In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image


In [2]:
# dataset file path

malignant_files = "dataset/trial/Malignant"
normal_files = "dataset/trial/Normal"

In [3]:
# function to load the images from the directories

def load_images(directory, label):
  images = []
  labels = []
  for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      img_path = os.path.join(directory, filename)
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
      if img is not None:
        img = cv2.resize(img, (128, 128))
        images.append(img)
        labels.append(label)
  return images, labels

In [4]:
# normal = 0
# malignant = 1

malignant_images, malignant_labels = load_images(malignant_files, label=1)
normal_images, normal_labels = load_images(normal_files, label=0)

In [5]:
images = malignant_images + normal_images
labels = malignant_labels + normal_labels

In [6]:
images = np.array(images)
labels = np.array(labels)


print(len(images))

971


In [7]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
categories = {
    "Malignant": malignant_files,
    "Normal": normal_files
}

In [9]:
# Model Architecture
model = Sequential([
    Input(shape=(128*128,)),  # Flattened input shape (128x128 grayscale image)

    # Fully Connected Layers
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),

    # Output Layer
    Dense(2, activation='softmax')
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert images to 1D array and one-hot encode labels
x_train_flattened = x_train.reshape(-1, 128*128)
x_test_flattened = x_test.reshape(-1, 128*128)
y_train_encoded = to_categorical(y_train, num_classes=2)
y_test_encoded = to_categorical(y_test, num_classes=2)

# Train the Model with validation split
history = model.fit(
    x_train_flattened, y_train_encoded,  # Training data
    epochs=10,
    batch_size=32,
    validation_split=0.2  # Use 20% of the training data for validation
)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 171ms/step - accuracy: 0.4930 - loss: 1099.2683 - val_accuracy: 0.6667 - val_loss: 25.8748
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.4984 - loss: 776.8760 - val_accuracy: 0.6474 - val_loss: 48.0157
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step - accuracy: 0.6011 - loss: 281.2659 - val_accuracy: 0.6795 - val_loss: 42.7831
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - accuracy: 0.6366 - loss: 127.9246 - val_accuracy: 0.7308 - val_loss: 17.5068
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.6570 - loss: 71.6951 - val_accuracy: 0.8141 - val_loss: 8.6994
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - accuracy: 0.7052 - loss: 33.9512 - val_accuracy: 0.7115 - val_loss: 27.0233
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.6557 - loss: 33.5500 - val_accuracy: 0.8205 - val_loss: 0.6191
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.6719 - loss: 2.8875 -

In [12]:
y_test_encoded = to_categorical(y_test, num_classes=2)
# Reshape x_test to match the input shape expected by the model
x_test_flattened = x_test.reshape(-1, 128*128)
x_test_flattened = x_test_flattened.astype('float32')

test_loss, test_acc = model.evaluate(x_test_flattened, y_test_encoded)
#print(f'Test Loss: {test_loss}')
#print(f'Test Accuracy: {test_acc}')

print("Model Accuracy =",test_acc*100)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5898 - loss: 0.6237
Model Accuracy = 57.94872045516968
